In [1]:
import pandas as pd
import numpy as np

In [2]:
(clusters, clusters_sim, is_main_label, uri_to_label) = pd.read_pickle('temp/clusters.pkl.bz2')

In [3]:
rows = []

for c in range(len(clusters)):
    for i in range(len(clusters[c])):
        v = np.delete(clusters_sim[c][i], i)
        rows.append((c, i, clusters[c][i], v.std(), v.mean(), np.median(v)))

/home/piotr/anaconda3/envs/iccs2022_clean/lib/python3.9/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/piotr/anaconda3/envs/iccs2022_clean/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/piotr/anaconda3/envs/iccs2022_clean/lib/python3.9/site-packages/numpy/core/_methods.py:256: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/tmp/ipykernel_11300/2612995492.py:6: RuntimeWarning: Mean of empty slice.
  rows.append((c, i, clusters[c][i], v.std(), v.mean(), np.median(v)))
/home/piotr/anaconda3/envs/iccs2022_clean/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/piotr/anaconda3/envs/iccs2022

In [4]:
df_main = pd.DataFrame.from_dict(is_main_label, orient='index', columns=['is_main_label'])
df_label = pd.DataFrame.from_dict(uri_to_label, orient='index', columns=['label'])

In [5]:
df = pd.DataFrame(rows, columns=['cluster', 'i', 'uri', 'std', 'mean', 'median']).set_index('uri')
df = df.join(df_main).join(df_label)

In [6]:
grouped = df[['cluster', 'i', 'mean', 'std']].groupby('cluster').agg({
    'i': 'count',
    'mean': 'mean',
    'std': 'mean',
})
grouped['score'] = grouped['mean'] + 2 * grouped['std']

In [7]:
sus1 = grouped[grouped.i > 3].sort_values(by='score')[:100]
sus1_g = df.join(sus1, on='cluster', rsuffix='_agg', how='right')

In [8]:
sus2 = grouped[grouped.i == 3].sort_values(by='score')[:15]
sus2_g = df.join(sus2, on='cluster', rsuffix='_agg', how='right')

In [9]:
final_suspects = sus1_g.append(sus2_g)
final_suspects.to_csv('results/final_suspects.csv')